## Define Constants

In [6]:
import torch

MAX_MEMORY = 500_000  # Increase memory buffer for better long-term learning
BATCH_SIZE = 512  # L4 can handle larger batch sizes efficiently
LEARNING_RATE = 0.0025  # Reduce LR for more stable convergence
GAMMA = 0.99  # Higher discount factor for long-term rewards
EPSILON_DECAY = 0.995  # Slower decay to improve exploration
MIN_EPSILON = 0.05  # Avoid premature exploitation

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == 'cuda':
    torch.backends.cudnn.benchmark = True # Optimize CUDA operations

## Define Environment

In [7]:
import pandas as pd
import numpy as np
from enum import Enum
import torch
from typing import Tuple, List

class Action(Enum):
    HOLD = 0
    LONG = 1
    SHORT = 2
    FLATTEN = 3

class TradeSide(Enum):
    LONG = 0
    SHORT = 1

class Position:
    def __init__(self, entry_price: float, size: int, side: TradeSide):
        self.entry_price = entry_price
        self.size = size
        self.side = side

    def get_unrealized_pnl(self, current_price: float) -> float:
        diff = (current_price - self.entry_price) if self.side == TradeSide.LONG else (self.entry_price - current_price)
        return self.entry_price * self.size * (diff / ((self.entry_price + current_price) / 2) / 100)

    def close(self, current_price: float) -> float:
        return self.get_unrealized_pnl(current_price)

class TradingEnvironment:
    REQUIRED_FEATURES = [
        "Asset1_Price", "Asset2_Price", "Ratio_Price", "Spread_ZScore", "Rolling_Correlation",
        "Rolling_Cointegration_Score", "RSI1", "RSI2", "RSI3",
        "MACD1", "MACD2", "MACD3"
    ]

    PERFORMANCE_COLUMNS = ["Unrealized_PnL", "Realized_PnL", "Positioned"]

    def __init__(self, df: torch.Tensor, plot: bool = False, debug: bool = False):
        self.step = 0
        self.window_size = 1
        self.debug = debug
        self.plot = plot
        self.balance = self.equity = 1_000_000
        self.positions: List[Tuple[Position, Position]] = []
        self.realized_pnl = self.unrealized_pnl = 0
        self.trade_history, self.reward_history = [], []
        self.buy_signals, self.sell_signals = [], []
        self.losing_streak, self.winning_streak = 0, 0
        self.last_trade_step = 0
        self.data = df
        self._prepare_data()

    def _prepare_data(self):
      empty_column = torch.zeros(self.data.shape[0], 1).to(device)
      for feature in self.PERFORMANCE_COLUMNS:
        self.data = torch.cat((self.data, empty_column), dim=1)
      print(self.data.shape)

    def reset(self) -> torch.Tensor:
        self.step = self.realized_pnl = self.unrealized_pnl = 0
        self.balance = self.equity = 1_000_000
        self.positions.clear()
        self.trade_history.clear()
        self.reward_history.clear()
        self.buy_signals.clear()
        self.sell_signals.clear()
        self.last_trade_step = 0
        return self.current_observation()

    def step_forward(self, action: Action) -> Tuple[torch.Tensor, float, float, float, bool]:
        self.step += 1

        reward, profit, positioned = self.execute_trade(action)
        self.trade_history.append(self.realized_pnl)
        self.reward_history.append(reward)

        if self.data is None:
            if self.debug:
                print("DEBUG: self.data is None in step_forward")
            done = True
        else:
            done = self.step + self.window_size >= len(self.data)

        obs = self.current_observation()
        obs[13] = self.unrealized_pnl
        obs[14] = self.realized_pnl
        obs[15] = positioned
        return obs, reward, self.realized_pnl, self.unrealized_pnl, done

    def execute_trade(self, action: Action) -> Tuple[float, float, int]:
        asset1_price = self.data[self.step, 0].item()
        asset2_price = self.data[self.step, 1].item()
        hedge_ratio = self.data[self.step, 3].item()

        reward = 0
        size = 100_000
        profit = 0

        if action == Action.HOLD:
            self.update_unrealized_pnl()
            self.last_trade_step += 1

        elif action in (Action.LONG, Action.SHORT):
            positions = (
                Position(asset1_price, size, TradeSide.LONG if action == Action.LONG else TradeSide.SHORT),
                Position(asset2_price, int(size * hedge_ratio), TradeSide.SHORT if action == Action.LONG else TradeSide.LONG)
            )
            self.positions.append(positions)
            (self.buy_signals if action == Action.LONG else self.sell_signals).append(self.step)
            self.last_trade_step = 0
            if len(positions) > 2:
                reward = self.calculate_reward(self.realized_pnl, self.unrealized_pnl, True, True)
            elif len(positions) == 2:
                reward = self.calculate_reward(self.realized_pnl, self.unrealized_pnl, True, False)

        elif action == Action.FLATTEN and self.positions:
            pnl = 0
            for positions in self.positions:
                pnl += positions[0].close(asset1_price) + positions[1].close(asset2_price)
                profit += pnl
                self.realized_pnl += pnl
            self.unrealized_pnl = 0
            self.positions.clear()
            reward = self.calculate_reward(pnl, self.unrealized_pnl, False, False)

        return reward, profit, int(bool(self.positions))

    def update_unrealized_pnl(self):
      if not self.positions:
          self.unrealized_pnl = 0
          return

      self.unrealized_pnl = sum(
          pos_a.get_unrealized_pnl(self.data[self.step, 0].item()) + pos_b.get_unrealized_pnl(self.data[self.step, 1].item())
          for pos_a, pos_b in self.positions
      )

    def calculate_reward(
        self,
        rpnl: float,
        upnl: float,
        entry: bool,
        dca: bool = False,
        transaction_costs: float = 0.001,
        risk_free_rate: float = 0.02,
        decay_factor: float = 0.99
    ):
        """
        Enhanced reward function with smooth normalization for statistical arbitrage.

        Parameters:
        - rpnl: Realized Profit and Loss
        - upnl: Unrealized Profit and Loss
        - entry: Whether this is a trade entry point
        - transaction_costs: Cost per trade
        - risk_free_rate: Benchmark risk-free rate for comparison
        - decay_factor: Decay factor for historical rewards to prevent reward inflation

        Returns:
        - Normalized reward score
        """
        if not hasattr(self, '_reward_history'):
            self._reward_history = []

        reward = 0.0
        net_profit = rpnl - transaction_costs
        profit_reward = np.sign(net_profit) * np.log1p(abs(net_profit))
        reward += profit_reward

        if entry:
            if dca:
              reward -= 1
            lookahead_steps = min(30, len(self.data) - self.step)
            lookforward1 = self.data[self.step:self.step + lookahead_steps, 0]
            lookforward2 = self.data[self.step:self.step + lookahead_steps, 1]

            entry_mfe1 = max(lookforward1.max() - self.data[self.step, 0], 0)
            entry_mae1 = min(lookforward1.min() - self.data[self.step, 0], 0)
            entry_mfe2 = max(lookforward2.max() - self.data[self.step, 1], 0)
            entry_mae2 = min(lookforward2.min() - self.data[self.step, 1], 0)

            entry_mfe1_cpu = entry_mfe1.cpu().detach().numpy()
            entry_mae1_cpu = entry_mae1.cpu().detach().numpy()

            entry_mfe2_cpu = entry_mfe2.cpu().detach().numpy()
            entry_mae2_cpu = entry_mae2.cpu().detach().numpy()

            entry_quality1 = np.arctan(entry_mfe1_cpu - abs(entry_mae1_cpu))
            entry_quality2 = np.arctan(entry_mfe2_cpu - abs(entry_mae2_cpu))
            reward += 0.5 * (entry_quality1 + entry_quality2)
        elif not entry:
            streak_component = 0
            if rpnl > 10:
                self.winning_streak += 1
                self.losing_streak = 0
            else:
                self.losing_streak += 1
                self.winning_streak = 0

            if self.losing_streak > 3:
                streak_component = self.losing_streak * -0.6
            elif self.winning_streak > 3:
                streak_component = self.winning_streak * 0.7
            reward += streak_component

        self.update_unrealized_pnl()
        if self.unrealized_pnl < 100:
            reward -= 1
        elif self.unrealized_pnl > 200:
            reward += 1

        self._reward_history.append(reward)
        final_reward = reward
        return final_reward

    def current_observation(self) -> torch.Tensor:
      # Check if self.data is None
      if self.data is None:
          print("DEBUG: self.data is None in current_observation")
          # Return a tensor of zeros with the shape of REQUIRED_FEATURES + PERFORMANCE_COLUMNS
          return torch.zeros(len(self.REQUIRED_FEATURES) + len(self.PERFORMANCE_COLUMNS))

      # Check if step is out of bounds
      elif self.step >= len(self.data):
          print(f"DEBUG: Step index {self.step} is out of bounds for data length {len(self.data)}")
          # Return the last observation in self.data
          return self.data[-1].clone()  # Use .clone() to return a copy, similar to pandas `.copy()`

      else:
          # Return the observation at the current step
          return self.data[self.step].clone()


## Define Neural Network

In [8]:
import torch.nn as nn
import torch.nn.functional as F
import os

class LSTM_Q_Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.3, bidirectional=True):
        super(LSTM_Q_Net, self).__init__()
        self.hidden_size = hidden_size
        self.bidirectional = bidirectional

        # Bidirectional LSTM (if applicable)
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=2,  # Multiple layers of LSTM
            batch_first=True,
            dropout=dropout,
            bidirectional=bidirectional,
        )

        # Fully connected layers
        self.fc1 = nn.Linear(hidden_size * (2 if bidirectional else 1), 128)
        self.fc2 = nn.Linear(128, output_size)

        # Initialize weights
        self._init_weights()

    def forward(self, x):
        # If the input is a 1D tensor (e.g., single time step), reshape it
        if len(x.shape) == 1:  # (input_size,)
            x = x.unsqueeze(0).unsqueeze(0)  # Add batch and sequence dimensions -> (1, 1, input_size)

        # LSTM expects input shape: (batch_size, sequence_length, input_size)
        lstm_out, _ = self.lstm(x)
        last_step = lstm_out  # Get the output of the last time step

        # Pass through fully connected layers
        x = F.sigmoid(self.fc1(last_step))
        return self.fc2(x)

    def _init_weights(self):
        # Proper weight initialization for LSTM and fully connected layers
        for name, param in self.named_parameters():
            if 'weight' in name:
                if 'lstm' in name:
                    # Xavier initialization for LSTM weights
                    nn.init.xavier_uniform_(param)
                else:
                    # Xavier initialization for fully connected layer weights
                    nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                # Initialize biases to zero
                nn.init.zeros_(param)

    def save(self, file_name):
        model_folder_path = "./models"
        os.makedirs(model_folder_path, exist_ok=True)
        file_path = os.path.join(model_folder_path, file_name)
        torch.save(self.state_dict(), file_path)


## Define Trainer

In [9]:
from torch import optim
class QTrainer:
    def __init__(self, model, lr, gamma, batch_size, target_update_freq=50):
        self.lr = lr
        self.gamma = gamma
        self.model = model.to(device)

        # Create target network with same architecture
        self.target_model = LSTM_Q_Net(
            input_size=16,
            hidden_size=model.hidden_size,
            output_size=model.fc2.out_features,
            bidirectional=model.bidirectional
        ).to(device)

        self.update_target()

        # Use Adam with improved parameters
        self.optimizer = optim.Adam(model.parameters(), lr=self.lr, amsgrad=True)

        # Less frequent LR adjustments
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=20, gamma=0.5)

        # Huber loss (SmoothL1Loss) is more robust for RL
        self.criterion = nn.SmoothL1Loss()

        self.batch_size = batch_size
        self.target_update_freq = target_update_freq
        self.train_step_count = 0

    def update_target(self):
        self.target_model.load_state_dict(self.model.state_dict())

    def train_step(self, state, action, reward, next_state, done):
        # Reshape action for gathering
        action = action.view(-1, 1)

        # Get current Q values
        pred = self.model(state)

        # Implement Double Q-learning for stability
        with torch.no_grad():
            # Get actions from main network
            next_actions = self.model(next_state).argmax(dim=1, keepdim=True)

            # Get Q-values from target network
            next_q_values = self.target_model(next_state).gather(1, next_actions)

            # Compute target Q values
            target = reward.unsqueeze(1) + (1 - done.float().unsqueeze(1)) * self.gamma * next_q_values

        # Get Q values for taken actions
        q_values = pred.gather(1, action)

        # Calculate loss and optimize
        loss = self.criterion(q_values, target)
        self.optimizer.zero_grad()
        loss.backward()

        # Apply gradient clipping
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)

        self.optimizer.step()

        # Update target network less frequently
        self.train_step_count += 1
        if self.train_step_count % self.target_update_freq == 0:
            self.update_target()
            self.scheduler.step()

## Define Training Agent

In [10]:
import numpy as np
import torch
from torch import optim
from collections import deque
from tqdm.notebook import tqdm
from typing import Tuple

class TradingAgent:
    def __init__(self, data: torch.Tensor, plot: bool = False, debug: bool = False):
        self.debug = debug
        self.epsilon = 0.25
        self.memory = deque(maxlen=MAX_MEMORY)
        self.data = data
        self.model = LSTM_Q_Net(input_size=16, hidden_size=128, output_size=4).to(device)
        self.trainer = QTrainer(self.model, lr=LEARNING_RATE, gamma=GAMMA, batch_size=BATCH_SIZE)
        self.env = TradingEnvironment(data, plot, debug)
        self.equity_curves = {}

    def store_experience(self, state, action, reward, next_state, done):
        """Stores experiences with clipped rewards."""
        if reward is None or np.isnan(reward):
            if self.debug:
                print("DEBUG: Received None or NaN reward, setting to 0.0")
            reward = 0.0
        reward = np.clip(reward, -1, 1)
        self.memory.append((state, action, reward, next_state, done))

    def sample_experiences(self):
        """Priority experience replay with absolute reward-based weighting."""
        if len(self.memory) < BATCH_SIZE:
            return list(self.memory)

        rewards = np.array([exp[2] for exp in self.memory], dtype=np.float32)
        priorities = np.abs(rewards) + 1e-6  # Avoid zero probabilities
        probabilities = priorities / np.sum(priorities)

        indices = np.random.choice(len(self.memory), BATCH_SIZE, p=probabilities, replace=False)
        return [self.memory[i] for i in indices]

    def update_epsilon(self):
        """Decay epsilon for exploration-exploitation balance."""
        self.epsilon = max(MIN_EPSILON, self.epsilon * EPSILON_DECAY)

    def select_action(self, state: torch.Tensor) -> Tuple[Action, int]:
        """Selects an action using epsilon-greedy strategy."""
        if np.random.rand() < self.epsilon:
            action_idx = np.random.randint(0, 4)
        else:
            with torch.no_grad():
                action_idx = self.model(state.unsqueeze(0)).argmax().item()

        return [Action.HOLD, Action.LONG, Action.SHORT, Action.FLATTEN][action_idx], action_idx

    def train(self):
        """Trains the model using batched experience replay."""
        batch = self.sample_experiences()
        states, actions, rewards, next_states, dones = zip(*batch)

        # Convert to tensors
        states = torch.stack(states).to(device)
        next_states = torch.stack(next_states).to(device)
        actions = torch.tensor(actions, dtype=torch.long, device=device).unsqueeze(1)
        rewards = torch.tensor(rewards, dtype=torch.float32, device=device).unsqueeze(1)
        dones = torch.tensor(dones, dtype=torch.bool, device=device).unsqueeze(1)

        return self.trainer.train_step(states, actions, rewards, next_states, dones)

    def save_model(self, episode: int):
        """Saves the trained model with error handling."""
        try:
            filename = f"Episode-{episode}.pth" if episode is not None else "Episode-unknown.pth"
            self.model.save(filename)
            if self.debug:
                print(f"DEBUG: Model saved as {filename}")
        except Exception as e:
            print(f"DEBUG: Model save failed - {str(e)}")

    def run(self, episodes: int):
        """Runs training episodes with performance tracking."""
        for episode in tqdm(range(episodes), desc="Training Progress", position=0):
            state, done = self.env.reset(), False
            equity_curve, episode_rewards, episode_losses = [], [], []
            total_steps, max_drawdown, peak_equity = 0, 0, 0

            with tqdm(total=len(self.data), desc=f"Episode {episode + 1}", leave=False, position=1) as step_bar:
                while not done:
                    action, action_idx = self.select_action(state)
                    next_state, reward, real_profit, _, done = self.env.step_forward(action)

                    equity_curve.append(real_profit)
                    episode_rewards.append(reward)
                    total_steps += 1

                    peak_equity = max(peak_equity, real_profit)
                    max_drawdown = max(max_drawdown, (peak_equity - real_profit) / (peak_equity + 1e-8))

                    self.store_experience(state, action_idx, reward, next_state, done)
                    loss = self.train()
                    if loss is not None:
                        episode_losses.append(loss.item())

                    state = next_state
                    step_bar.update(1)

            self.equity_curves[episode] = equity_curve

            # Episode summary
            print({
                'episode': episode,
                'total_steps': total_steps,
                'total_reward': np.sum(episode_rewards),
                'mean_reward': np.mean(episode_rewards) if episode_rewards else 0,
                'max_drawdown': max_drawdown,
                'last_loss': episode_losses[-1] if episode_losses else None
            })

            self.update_epsilon()
            self.save_model(episode)


## Initialize

In [11]:
data = np.genfromtxt('data/processed/train_data_scaled.csv', delimiter=',', skip_header=False, dtype=None, names=True, encoding=None)
print(f"First few rows of data: {data[:5]}")
print(f"Column names: {data.dtype.names}")
data_removed = np.column_stack([data[field] for field in data.dtype.names[1:]])
data_tensor = torch.tensor(data_removed, dtype=torch.float32).to(device)
print(f"Shape of the tensor: {data_tensor.shape}")

agent = TradingAgent(data_tensor, plot=False, debug=False)


First few rows of data: [('2016-01-04 00:29:00', 0.43808314, 0.93958347, 0.02263698, 0.41702082, 0.33888563, 0.91981418, 1., 0.26679839, 0.38031375, 0.35007507, 0.51645163, 0.73697773, 0.29400669)
 ('2016-01-04 00:30:00', 0.43791593, 0.93928257, 0.02280172, 0.41696028, 0.40237618, 0.93247409, 1., 0.25321426, 0.32934684, 0.38543971, 0.51526629, 0.7364399 , 0.29387281)
 ('2016-01-04 00:31:00', 0.43808314, 0.93939003, 0.02284263, 0.41721442, 0.4067852 , 0.93922893, 1., 0.29190755, 0.36218071, 0.39425633, 0.51481955, 0.73618861, 0.29387499)
 ('2016-01-04 00:32:00', 0.43851787, 0.93923958, 0.02340607, 0.41651643, 0.53328086, 0.93713808, 1., 0.38164029, 0.33724664, 0.50055925, 0.51547785, 0.73584345, 0.29464702)
 ('2016-01-04 00:33:00', 0.43922015, 0.93930406, 0.02398924, 0.41545451, 0.63931675, 0.92454615, 1., 0.49338477, 0.3576291 , 0.58232302, 0.51751565, 0.73569638, 0.29603847)]
Column names: ('Gmt_time', 'Asset1_Price', 'Asset2_Price', 'Ratio_Price', 'Hedge_Ratio', 'Spread_ZScore', 'Rol

## Run Training Algorithm

In [12]:
agent.run(50)

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html